#### Names of people in the group

Please write the names of the people in your group in the next cell.

Jacob Ørmen Clements

In [0]:
# Loading modules that we need
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Add your imports below this line

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame that I have altered to work with my code from task 1
def load_df(table_name: str) -> DataFrame:
    # Match the path with the one used in the save_df function
    path = f"dbfs:/user/hive/warehouse/{table_name}.parquet"
    return spark.read.parquet(path)

users_df = load_df("users")
posts_df = load_df("posts")

#### The Problem: Mining the Interests of Experts

In [0]:
# Defining variable A "Expertise level"

# Select the 'Id' and 'Reputation' columns from the users DataFrame
expertise_df = users_df.select('Id', 'Reputation')

# Renaming the 'Id' column to 'UserId' for clarity during join operations
expertise_df = expertise_df.withColumnRenamed('Id', 'UserId')

expertise_df.show()

+------+----------+
|UserId|Reputation|
+------+----------+
|    -1|         1|
|     1|       101|
|     2|       101|
|     3|       101|
|     4|       101|
|     5|       215|
|     6|       101|
|     7|       101|
|     8|       101|
|     9|      1102|
|    10|       101|
|    11|       213|
|    12|       101|
|    14|      2782|
|    15|       101|
|    16|         1|
|    17|       236|
|    18|       101|
|    19|       101|
|    20|       101|
+------+----------+
only showing top 20 rows



In [0]:
# Defining variable B "user interest diversity"

# Filter out answers from the posts DataFrame
answers_df = posts_df.filter(posts_df.PostTypeId == 2)

# Join answers to their parent questions to get the tags
# We'll need to alias the dataframes to differentiate between columns with the same name
answers_with_questions_df = answers_df.alias("ans")\
    .join(posts_df.alias("ques"), col("ans.ParentId") == col("ques.Id"))\
    .select(col("ans.OwnerUserId"), col("ques.Tags"))

# Extract tags from the string and explode the array to have one tag per row
tagged_answers_df = answers_with_questions_df\
    .withColumn("Tag", explode(split(col("ques.Tags"), "<|>")))\
    .filter(col("Tag") != "")

# Group by user and count distinct tags
user_tags_df = tagged_answers_df.groupBy("OwnerUserId")\
    .agg(countDistinct(col("Tag")).alias("DistinctTagCount"))

# Normalize the distinct tag count by the total number of unique tags (638)
user_interest_diversity_df = user_tags_df\
    .withColumn("InterestDiversity", col("DistinctTagCount") / lit(638))

# Show the DataFrame to verify the results
user_interest_diversity_df.show()


+-----------+----------------+--------------------+
|OwnerUserId|DistinctTagCount|   InterestDiversity|
+-----------+----------------+--------------------+
|        471|              56|  0.0877742946708464|
|      45615|              10| 0.01567398119122257|
|      57693|               5|0.007836990595611285|
|      69478|              66| 0.10344827586206896|
|        463|               3|0.004702194357366771|
|      31951|               4|0.006269592476489028|
|      29054|               4|0.006269592476489028|
|      13285|              16|0.025078369905956112|
|     108560|               4|0.006269592476489028|
|      49717|               5|0.007836990595611285|
|      80579|               3|0.004702194357366771|
|      57178|               8|0.012539184952978056|
|       9465|              11|0.017241379310344827|
|      31528|               4|0.006269592476489028|
|      50223|               5|0.007836990595611285|
|      14450|               4|0.006269592476489028|
|      74852

Now that we have both variables A and B the next steps are to:

Join expertise_df with user_interest_diversity_df on the UserId/OwnerUserId column to combine the expertise level and interest diversity measures into one DataFrame. After this we can calculate the Pearson correlation coefficient between the Reputation and Interest Diversity columns in the joined DataFrame.

In [0]:
# Join expertise_df with user_interest_diversity_df
combined_df = expertise_df.join(user_interest_diversity_df, expertise_df.UserId == user_interest_diversity_df.OwnerUserId)

# Select only the necessary columns for calculation
data_for_correlation = combined_df.select('Reputation', 'InterestDiversity')

# Calculate the Pearson correlation coefficient (this time using DataFrame helper functions)
correlation = data_for_correlation.stat.corr('Reputation', 'InterestDiversity')

print("Pearson correlation coefficient:", correlation)

Pearson correlation coefficient: 0.721767764862296



The Pearson correlation coefficient calculated is approximately 0.721, which indicates a fairly strong positive linear relationship between the user's reputation (expertise level) and diversity. The positive correlation suggests that as the reputation of a user increases, the diversity of the topics they engage with also increases. This means that users with higher reputations tend to answer a wider variety of questions across different tags.